In [1]:
import sys
import os

PROJECT_DIR = '/home/evgenshuben/Desktop/gitReps/YandexCup/'
sys.path.append(PROJECT_DIR)
os.chdir(PROJECT_DIR)

In [2]:
from omegaconf import OmegaConf
import torch.nn as nn

In [3]:
%load_ext autoreload
%autoreload 2

from scr.train import Trainer

In [4]:
# Путь к конфигурационному файлу, сохраненному Hydra
config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-3/hydra/.hydra/config.yaml"



cfg = OmegaConf.load(config_path)

In [5]:
trainer = Trainer(cfg)

/home/evgenshuben/Desktop/gitReps/YandexCup/scr/train.py:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(self.cfg.pipeline.model_ckpt

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [9]:
for batch in trainer.train_dataloader:
    break

In [31]:
class ResCNNBlock1d(nn.Module):
    def __init__(self, in_channels, out_channels, pool=True):
        super(ResCNNBlock1d, self).__init__()

        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1, stride=1, bias=False)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(2) if pool else nn.Identity()

        # Слой для проекции, если количество входных каналов не совпадает с выходными
        self.shortcut = nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=1, padding=0) if in_channels != out_channels else nn.Identity()

    def forward(self, x):
        residual = self.shortcut(x)  # Применяем проекцию к входу
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)

        # Применяем пропускную связь
        x += residual  # Добавляем вход к выходу блока
        x = self.relu(x)  # Применяем активацию после добавления

        x = self.maxpool(x)
        return x

In [41]:
class ResNet1d(nn.Module):
    def __init__(self, emb_size=512, num_classes=8858, input_norm=False):
        super(ResNet1d, self).__init__()

        self.layer_norm = nn.LayerNorm([84, 50]) if input_norm else nn.Identity()
        self.block1 = ResCNNBlock1d(84, 64)
        self.block2 = ResCNNBlock1d(64, 128)
        self.block3 = ResCNNBlock1d(128, 256)
        self.block4 = ResCNNBlock1d(256, 512)  # Добавлен новый блок для увеличения глубины
        self.adaptive_pool = nn.AdaptiveAvgPool1d(1)
        self.fc_emb = nn.Linear(512, emb_size)  # Увеличиваем вход для линейного слоя
        self.fc = nn.Linear(emb_size, num_classes)

    def forward(self, x):
        x = self.layer_norm(x)
        
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)  # Новый блок

        pooled_output = self.adaptive_pool(x)
        pooled_output = pooled_output.view(pooled_output.size(0), -1)

        emb = self.fc_emb(pooled_output)
        cls = self.fc(emb)

        return dict(emb=emb, cls=cls)

In [42]:
model = ResNet1d()
model.eval()

ResNet1d(
  (layer_norm): Identity()
  (block1): ResCNNBlock1d(
    (conv1): Conv1d(84, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (shortcut): Conv1d(84, 64, kernel_size=(1,), stride=(1,))
  )
  (block2): ResCNNBlock1d(
    (conv1): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): R

In [43]:
for batch in trainer.train_dataloader:
    break

In [44]:
batch['anchor'].shape

torch.Size([200, 84, 50])

In [47]:
model(batch['anchor'])['emb'].shape

torch.Size([200, 512])

In [48]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
DataLoader()